In [ ]:
# setup
import teradata
import pandas as pd
import scipy as sp
from scipy import sparse
#from scipy import linalg
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
udaExec = teradata.UdaExec ()
np.set_printoptions(threshold=np.nan)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# get the source data tables from Teradata
with udaExec.connect("${dataSourceName}") as session: 
    tran = pd.DataFrame(session.execute("""SELECT id0-1, id1-1, Rate, GR FROM sandbox.adb_cltv_v where id1 is not null""").fetchall())
    rev = pd.DataFrame(session.execute("""SEL id0-1, 0, AVG(gr) AS gr, seg FROM  sandbox.adb_cltv_v GROUP BY 1,2,4 order by 1""").fetchall()) 

In [ ]:
# build transition and payoff matrices
tmat = sp.sparse.coo_matrix((tran[2],(tran[0],tran[1])),shape=(1800,1800),dtype="d").todense()
pmat = sp.sparse.coo_matrix((rev[2],(rev[0],rev[1])),shape=(1800,1),dtype="d").todense()

In [ ]:
# period is monthly for WACC
def monthly_wacc(x): return (1+x)**(1/12)

# sum to infinity of Markov transition model
def cltv_calc(weighted_av_cost_capital, transition_matrix, payoff_matrix): 
    return (
            (np.identity(1800)-((monthly_wacc((wacc)))**-1)*transition_matrix).I
           ).dot(payoff_matrix)

# output array
wacc = 0.135
cltv_a = cltv_calc(wacc, tmat, pmat).A

In [ ]:
# rebuild frame and segment labels from the seg code in case I want to analyse more in here
cltv = pd.DataFrame(cltv_a, columns=['cltv'])
cltv['M0'] = pmat.A.astype(float64)
cltv['fCLTV'] = (cltv['cltv'] - cltv['M0']).astype(float64)
cltv['id0'] = cltv.index+1
cltv['segment'] = rev[3]
cltv['product'] = cltv['segment'].str[0]
cltv['promo'] = cltv['segment'].str[1]
cltv['recency'] = cltv['segment'].str[2:4].astype(int)
cltv['frequency'] = cltv['segment'].str[4:6]
cltv['monetary'] = cltv['segment'].str[6:8]
lol = cltv.loc[:,['segment','fCLTV']].values.tolist()

In [ ]:
# upload the output cltv values back into Teradata
with udaExec.connect("${dataSourceName}") as session: 
    session.executemany("""insert into sandbox.adbCLTV (seg_code,CLTV) VALUES (?,?) """,lol,batch=True)

In [ ]:
# various views
#cltv.head(2)
#cltv.groupby(['recency','product']).mean().cltv
#pd.pivot_table(cltv, values='cltv', index=['recency'],columns=['product','monetary'],aggfunc=np.mean)
plt.scatter(cltv['recency'],cltv['fCLTV'],marker="+")

In [ ]:
cltv[cltv['fCLTV'] > 50]